Great! Let's start by building a **basic RAG (Retrieval-Augmented Generation) model** for a **trip itinerary planner**.  

### **📌 Plan for Building the RAG Model**
1️⃣ **Basic RAG Model**  
   - Use **FAISS** as the vector database  
   - Use **OpenAI (GPT-4) or Mistral** as the generator  
   - Store & retrieve trip-related documents  
   - Generate itinerary based on user query  

2️⃣ **Advanced RAG Enhancements**  
   - **Multi-hop Retrieval** (For complex queries)  
   - **Hybrid Retrieval** (BM25 + Dense embeddings)  
   - **Fine-tuning the retriever and LLM**  
   - **Multi-Modal RAG** (Text + Images for travel)  
   - **Streaming RAG** (Real-time updates on travel data)  

---



### **🛠 Step 1: Building a Basic RAG Model**
We will:  
✅ Load travel-related documents into FAISS  
✅ Use an embedding model (e.g., OpenAI, HuggingFace)  
✅ Retrieve relevant travel data based on user query  
✅ Generate trip plans with an LLM  

Let’s build this step by step.


In [ ]:
import os
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain



In [16]:
# Load API Key securely from .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI Chat Model (Updated API)
llm = ChatOpenAI(model="gpt-4", openai_api_key=api_key)

# Initialize OpenAI Embeddings (for Vector Search)
embeddings = OpenAIEmbeddings(openai_api_key=api_key)



In [17]:
# Sample Travel Information (Knowledge Base)
documents = [
    "Paris has beautiful landmarks like the Eiffel Tower and Louvre Museum.",
    "Tokyo is known for its cherry blossoms and sushi culture.",
    "New York has Times Square, Central Park, and Broadway shows.",
    "Rome is famous for its ancient Colosseum and Vatican City.",
    "The best time to visit Paris is in Spring or Fall.",
    "A 3-day itinerary for Paris includes visits to the Eiffel Tower, Louvre, and Seine River Cruise."
]



In [18]:
# Step 1: Create a Vector Database using FAISS
vectorstore = FAISS.from_texts(documents, embedding=embeddings)
retriever = vectorstore.as_retriever()



In [19]:
# Step 2: Define a Document Combination Chain (Fixing Previous Error)
qa_chain = load_qa_chain(llm, chain_type="stuff")  # "stuff" merges retrieved documents



In [20]:
# Step 3: Set up the Retrieval-Augmented Generation (RAG) Chain
rag_chain = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever)



In [22]:
# Step 4: User Query - Plan a trip
query = "Plan a 3-day trip to delhi"
response = rag_chain.run(query)

print("Generated Itinerary:\n", response)


Generated Itinerary:
 I'm sorry, but I don't have information on a 3-day itinerary for Delhi in the context provided.



### **✅ What This Does**
1. **Loads travel-related documents** (Basic info about cities).  
2. **Uses FAISS** to store & retrieve relevant travel data.  
3. **Embeds the documents using OpenAI embeddings**.  
4. **Uses GPT-4** to generate an itinerary.  

Try running this, and let me know if you want any modifications! 🚀  
Next, we’ll add **multi-hop retrieval** to handle more complex queries.

Let's **build Hybrid RAG + Multi-Hop RAG** step by step. 🚀  

---

### **🛠 Step 1: Hybrid Retrieval (BM25 + FAISS)**
🔹 **Why?**  
- **BM25 (Lexical)** retrieval is **good for keyword-based queries**.  
- **FAISS (Semantic)** retrieval is **good for meaning-based queries**.  
- **Hybrid RAG = Best of both worlds!**  

🔹 **How?**  
- Use **BM25** (`rank_bm25` library) for **keyword retrieval**.  
- Use **FAISS** for **vector retrieval**.  
- **Combine scores** (BM25 + FAISS) to return the best documents.

---

### **🛠 Step 2: Multi-Hop Retrieval (Answering Complex Queries)**
🔹 **Why?**  
- Some queries **require multiple retrieval steps**.  
- Example:  
  **“What are the best tourist spots in Paris, and how do I get there?”**  
  - **First step:** Find top tourist spots in Paris.  
  - **Second step:** Retrieve transport options to these spots.  

🔹 **How?**  
- **Break query into sub-queries** dynamically.  
- **Retrieve iteratively**, refining search results at each step.  

---



### **🚀 Hybrid + Multi-Hop RAG Implementation**
I'll now create a **Python script** combining **Hybrid Retrieval (BM25 + FAISS) + Multi-Hop Retrieval** for a **Trip Itinerary RAG**.  

🔹 **Tech Stack:**  
✅ **BM25** (Lexical Search)  
✅ **FAISS** (Semantic Search)  
✅ **LangChain** (RAG Framework)  
✅ **OpenAI GPT-4** (LLM)  
✅ **FastAPI** (for future deployment)  

Let's write the **Hybrid Multi-Hop RAG code now!** ⏳


In [32]:
import os
import faiss
import numpy as np
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from rank_bm25 import BM25Okapi
from dotenv import load_dotenv


In [33]:

# Load API Keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI Chat Model
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY)


In [34]:

# Load Documents (Trip Itinerary Data)
def load_documents(file_path="data/trip_itineraries.txt"):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return text_splitter.create_documents([text])


In [35]:

# Preprocess Documents
docs = load_documents()

# BM25 (Lexical Search)
tokenized_corpus = [doc.page_content.split() for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)

# FAISS (Vector Search)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(docs, embeddings)


In [36]:

# 🔹 Hybrid Retrieval (BM25 + FAISS)
def hybrid_retrieval(query, bm25, tokenized_corpus, vectorstore, docs, top_k=3):
    # BM25 Lexical Search
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_indices = np.argsort(bm25_scores)[::-1][:top_k]
    bm25_docs = [docs[i] for i in bm25_top_indices]

    # FAISS Semantic Search
    faiss_docs = vectorstore.similarity_search(query, k=top_k)

    # Merge Results (Handling Duplicate Documents)
    combined_docs = {doc.page_content: doc for doc in (bm25_docs + faiss_docs)}.values()
    return list(combined_docs)


In [37]:

# 🔹 Multi-Hop RAG
def multi_hop_rag(query, retriever, llm, num_hops=2):
    intermediate_query = query
    context = ""

    for hop in range(num_hops):
        retrieved_docs = retriever(intermediate_query)
        retrieved_texts = "\n".join([doc.page_content for doc in retrieved_docs])

        context += f"\n--- Retrieved Context (Hop {hop+1}) ---\n{retrieved_texts}\n"

        # Generate intermediate query refinement
        messages = [
            SystemMessage(content="You are an intelligent travel assistant helping plan trips."),
            HumanMessage(content=f"Given this information, refine the question: {intermediate_query}\n{context}")
        ]
        response = llm(messages)
        intermediate_query = response.content

    # Final Answer Generation
    messages = [
        SystemMessage(content="You are an intelligent travel assistant."),
        HumanMessage(content=f"Using this retrieved knowledge, answer the final query: {query}\n{context}")
    ]
    final_response = llm(messages)

    return final_response.content


In [38]:

# Hybrid Retriever Wrapper
def hybrid_retriever(query):
    return hybrid_retrieval(query, bm25, tokenized_corpus, vectorstore, docs)


In [39]:

# 🔥 Run Multi-Hop Hybrid RAG
query = "What are the best tourist spots in Paris, and how do I get there?"
response = multi_hop_rag(query, hybrid_retriever, llm, num_hops=2)
print("\nFinal Answer:", response)



Final Answer: The best tourist spots in Paris are the Eiffel Tower, which is best visited at sunset for a stunning view, the Louvre Museum, home to the Mona Lisa and best visited early in the morning to avoid crowds, and the Notre-Dame Cathedral, a Gothic masterpiece. Another great way to explore the city is by taking a Seine River Cruise. The fastest way to travel within Paris and reach these spots is by using the Metro.



### ✅ **Hybrid + Multi-Hop RAG Features**
1️⃣ **Hybrid Retrieval** → Uses **BM25** (Lexical) + **FAISS** (Semantic)  
2️⃣ **Multi-Hop Retrieval** → Iterates query refinement for **complex questions**  
3️⃣ **OpenAI GPT-4** → Generates answers based on retrieved context  


## 🔥 **Building a RAG with Online Search Integration**  
We’ll enhance our RAG model by integrating **online search tools** (DuckDuckGo, Google, or SerpAPI) to fetch **real-time data** and combine it with existing knowledge from vector search.

---

## **🔹 RAG with Online Search + Vector Database**
### **🛠️ Features of This RAG**
✅ **Hybrid Retrieval**: Uses both **web search** and **vector embeddings** for retrieval.  
✅ **Online Search Tools**: Fetches the latest information from the web using **DuckDuckGo, Google, or SerpAPI**.  
✅ **Multi-Hop Querying**: Dynamically refines and improves results iteratively.  
✅ **OpenAI GPT for Answer Generation**: Uses **GPT-4** to process and synthesize responses.  

---

### **🚀 Implementation**


In [53]:

import os
import requests
import faiss
import numpy as np
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage, SystemMessage
from rank_bm25 import BM25Okapi
from dotenv import load_dotenv


In [54]:

# Load API Keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_KEY")  # For Google Search (Optional)

# Initialize OpenAI Chat Model
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY)


In [55]:

# 🔹 Load Local Knowledge Base
def load_documents(file_path="data/knowledge_base.txt"):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return text_splitter.create_documents([text])

docs = load_documents()


In [56]:
docs

[Document(metadata={}, page_content='Best tourist destinations in Paris:\n1. Eiffel Tower – Iconic landmark with breathtaking city views.\n2. Louvre Museum – Home to the Mona Lisa and other masterpieces.\n3. Notre-Dame Cathedral – A stunning Gothic cathedral.\n4. Champs-Élysées – A famous avenue with luxury shops.\n5. Montmartre – A historic district known for its art scene.'),
 Document(metadata={}, page_content='Best vegetarian restaurants in New York:\n1. Superiority Burger – Famous for its vegetarian burgers.\n2. Dirt Candy – High-end restaurant with innovative vegetarian dishes.\n3. abcV – Organic, plant-based cuisine with a chic setting.\n4. Jajaja – Mexican-inspired vegan food.\n5. Le Botaniste – Belgian-style plant-based eatery.'),
 Document(metadata={}, page_content="How to travel in Paris:\n- Metro: The Paris Métro is the fastest way to get around.\n- Buses: Extensive bus network covering all areas.\n- Biking: Vélib' bike rental is a great option.\n- Walking: Many attractions

In [57]:

# 🔹 BM25 (Lexical Search)
tokenized_corpus = [doc.page_content.split() for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)

# 🔹 FAISS (Vector Search)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(docs, embeddings)


In [58]:

# 🔹 Online Search Retrieval (DuckDuckGo API)
def web_search_duckduckgo(query, num_results=3):
    search_url = f"https://api.duckduckgo.com/?q={query}&format=json"
    response = requests.get(search_url).json()
    results = response.get("RelatedTopics", [])[:num_results]
    return [res["Text"] for res in results if "Text" in res]


In [59]:

# 🔹 Google Search (Using SerpAPI) - Alternative
def web_search_google(query, num_results=3):
    url = "https://serpapi.com/search"
    params = {"q": query, "api_key": SERPAPI_KEY, "num": num_results}
    response = requests.get(url, params=params).json()
    return [res["snippet"] for res in response.get("organic_results", []) if "snippet" in res]


In [60]:

# 🔹 Hybrid Retrieval (BM25 + FAISS + Web Search)
def hybrid_retrieval(query, bm25, tokenized_corpus, vectorstore, docs, top_k=3):
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_indices = np.argsort(bm25_scores)[::-1][:top_k]
    bm25_docs = [docs[i] for i in bm25_top_indices]
    
    faiss_docs = vectorstore.similarity_search(query, k=top_k)
    web_results = web_search_duckduckgo(query, num_results=top_k)  # Replace with `web_search_google()` if needed

    # Merge Results (Handling Duplicate Documents)
    combined_docs = {doc.page_content: doc for doc in (bm25_docs + faiss_docs)}.values()
    combined_texts = "\n".join([doc.page_content for doc in combined_docs] + web_results)
    
    return combined_texts


In [61]:

# 🔹 Multi-Hop Online RAG
def multi_hop_rag(query, retriever, llm, num_hops=2):
    intermediate_query = query
    context = ""

    for hop in range(num_hops):
        retrieved_texts = retriever(intermediate_query)
        context += f"\n--- Retrieved Context (Hop {hop+1}) ---\n{retrieved_texts}\n"

        # Generate intermediate query refinement
        messages = [
            SystemMessage(content="You are a knowledgeable AI assistant that provides factually accurate answers."),
            HumanMessage(content=f"Given this information, refine the question: {intermediate_query}\n{context}")
        ]
        response = llm(messages)
        intermediate_query = response.content

    # Final Answer Generation
    messages = [
        SystemMessage(content="You are an AI assistant that provides the best answers by combining sources."),
        HumanMessage(content=f"Using this retrieved knowledge, answer the final query: {query}\n{context}")
    ]
    final_response = llm(messages)

    return final_response.content


In [62]:

# Hybrid Retriever Wrapper
def hybrid_retriever(query):
    return hybrid_retrieval(query, bm25, tokenized_corpus, vectorstore, docs)


In [64]:

# 🔥 Run Multi-Hop Online RAG
query = "Plan a 5 days trip to New York?"
response = multi_hop_rag(query, hybrid_retriever, llm, num_hops=2)
print("\nFinal Answer:", response)



Final Answer: Day 1: Start your trip with a visit to iconic landmarks such as the Statue of Liberty and Times Square. You can also explore Central Park and enjoy a leisurely stroll or a boat ride in the lake.

Day 2: Dedicate this day to exploring the world-renowned museums. Visit the Metropolitan Museum of Art in the morning and the Museum of Modern Art in the afternoon.

Day 3: Visit the Empire State Building for breathtaking city views. Later, you can explore the vibrant neighborhoods of SoHo and Greenwich Village.

Day 4: Spend the day exploring Brooklyn. Visit the Brooklyn Bridge, Brooklyn Botanic Garden, and enjoy the food scene at Smorgasburg.

Day 5: Dedicate your last day to shopping and leisure. Visit Fifth Avenue for luxury shopping. End your trip with a Broadway show in the evening.

Throughout your trip, you can dine at some of the best vegetarian restaurants in New York such as Superiority Burger, Dirt Candy, abcV, Jajaja, and Le Botaniste.

Remember to follow general tr



---

## **🔹 Enhancements in This RAG**
✅ **Online Search Integration**: Uses **DuckDuckGo API** (or **Google SerpAPI**) for real-time data.  
✅ **Multi-Hop Querying**: Iteratively refines search queries for better retrieval.  
✅ **Hybrid Retrieval**: Combines **BM25, FAISS, and Web Search** for comprehensive results.  
✅ **Improved OpenAI API Usage**: Ensures proper `ChatOpenAI` formatting.  

---
